In [81]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, Sequence, MetaData
from sqlalchemy import select, update
from sqlalchemy.orm import mapped_column, DeclarativeBase
from datetime import datetime

In [48]:
full_path=r'./data/PacificOcean.csv'

In [49]:
with open(full_path,'rb') as f:
    data=pd.read_csv(f,encoding='utf-8')
data.head()

,code,location,city,country,lat,lon
0,waka,Wakkanai,稚内港-北海道稚内市,Japan,45.41,141.69
1,abas,Abashiri,网走港-北海道网走市,Japan,44.02,144.29
2,hana,Hanasaki,北海道根室市,Japan,43.28,145.57
3,kush,Kushiro,北海道钏路市,Japan,42.98,144.37
4,hako,Hakodate,北海道函馆市,Japan,41.78,140.72


In [5]:
data.columns

Index(['code', 'location ', 'city', 'country', 'lat', 'lon'], dtype='object')

In [38]:
# 数据库的配置，配置借鉴自 django 的 settings 的结构
DATABASES = {
    'default': {
        'ENGINE': 'mysqldb',  # 数据库引擎
        'NAME': 'surge_global_sys',  # 数据库名
        'USER': 'root',  # 账号
        'PASSWORD': '123456',
        'HOST': '127.0.0.1',  # HOST
        'POST': 3306,  # 端口
        'OPTIONS': {
            "init_command": "SET foreign_key_checks = 0;",
        },
    },
}

### 尝试将行政区域信息提取并写入 tb:region_info 

In [39]:
DEFAULT_COUNTRY_INDEX = -1

In [73]:
class DbFactory:
    """
        数据库工厂
    """

    def __init__(self, db_mapping: str = 'default', engine_str: str = None, host: str = None, port: str = None,
                 db_name: str = None,
                 user: str = None,
                 pwd: str = None):
        """
            mysql 数据库 构造函数
        :param db_mapping:
        :param engine_str:
        :param host:
        :param port:
        :param db_name:
        :param user:
        :param pwd:
        """
        db_options = DATABASES.get(db_mapping)
        self.engine_str = engine_str if engine_str else db_options.get(
            'ENGINE')
        self.host = host if host else db_options.get('HOST')
        self.port = port if port else db_options.get('POST')
        self.db_name = db_name if db_name else db_options.get('NAME')
        self.user = user if user else db_options.get('USER')
        self.password = pwd if pwd else db_options.get('PASSWORD')
        # TypeError: Invalid argument(s) 'encoding' sent to create_engine(), using configuration MySQLDialect_mysqldb/QueuePool/Engine.  Please check that the keyword arguments are appropriate for this combination of components.
        self.engine = create_engine(
            f"mysql+{self.engine_str}://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}?charset=utf8",
            pool_pre_ping=True, future=True)
        self._session_def = sessionmaker(bind=self.engine)

    @property
    def Session(self) -> sessionmaker:
        if self._session_def is None:
            self._session_def = sessionmaker(bind=self.engine)
        return self._session_def()


In [41]:
class BaseMeta(DeclarativeBase):
    pass


class IIdModel(BaseMeta):
    __abstract__ = True
    id: Mapped[int] = mapped_column(primary_key=True)


class IDel(BaseMeta):
    """
        软删除 抽象父类
    """
    __abstract__ = True
    is_del: Mapped[int] = mapped_column(nullable=False, default=0)


class IModel(BaseMeta):
    """
        model 抽象父类，主要包含 创建及修改时间
    """
    __abstract__ = True
    gmt_create_time: Mapped[datetime] = mapped_column(default=datetime.utcnow)
    gmt_modify_time: Mapped[datetime] = mapped_column(default=datetime.utcnow)


In [42]:
class RegionInfo(IIdModel, IDel, IModel):
    """
        行政区划表
    """
    val_en: Mapped[str] = mapped_column()
    val_ch: Mapped[str] = mapped_column()
    pid: Mapped[int] = mapped_column(default=-1)
    # location: Mapped[str] = mapped_column()
    # city: Mapped[str] = mapped_column()
    # city_name_ch: Mapped[str] = mapped_column()
    # country: Mapped[int] = mapped_column(nullable=False, default=DEFAULT_COUNTRY_INDEX)
    __tablename__ = 'region_info'

In [80]:
class StationInfo(IIdModel, IDel, IModel):
    station_name: Mapped[str] = mapped_column(default='DEFAULT')
    station_code: Mapped[str] = mapped_column(default='DEFAULT')
    lat: Mapped[float] = mapped_column(nullable=True)
    lon: Mapped[float] = mapped_column(nullable=True)
    desc: Mapped[str] = mapped_column()
    is_abs: Mapped[int] = mapped_column(nullable=False, default=0)
    # 所属父级 id
    pid: Mapped[int] = mapped_column(default=0)
    is_in_common_use: Mapped[int] = mapped_column(nullable=False, default=0)
    sort: Mapped[int] = mapped_column(nullable=False, default=0)
    # 归属的行政区划id tb: RegionInfo
    rid: Mapped[int] = mapped_column(default=0)
    __tablename__ = 'station_info'


In [16]:
num_columns=data.shape[1]
num_columns

6

In [21]:
num_rows=data.shape[0]
num_rows

173

In [43]:
data.iloc[0]

code               waka
location       Wakkanai
city         稚内港-北海道稚内市
country           Japan
lat               45.41
lon              141.69
Name: 0, dtype: object

In [50]:
type(data.iloc[0])

pandas.core.series.Series

In [52]:
data.iloc[0].loc['location']

'Wakkanai'

In [53]:
set(data['country'].tolist())

{'Australia',
 'CostaRica',
 'ElSalvador',
 'FijiIslands',
 'France',
 'IndependentStateofWesternSamoa',
 'Japan',
 'Mexico',
 'NewZealand',
 'Nicaragua',
 'PapuaNewGuinea',
 'Philippines',
 'PuertoRico',
 'SolomonIslands',
 'SouthernKorea',
 'USA',
 'USASamoa',
 'Vanuatu',
 'VietNam'}

In [74]:
session = DbFactory().Session


### step1:录入对应的国家

In [26]:
try:
    for country in set(data['country'].tolist()):
        region_model = RegionInfo(val_en=country, val_ch='')
        session.add(region_model)
    session.commit()

except Exception as ex:
    print(ex.args)
finally:

    session.close()


### step2:根据国家字典录入对应不同的城市

In [55]:
dict_country = {
    'SouthernKorea': 1,
    'PuertoRico': 2,
    'FijiIslands': 3,
    'France': 4,
    'SolomonIslands': 5,
    'USASamoa': 6,
    'Nicaragua': 7,
    'Australia': 8,
    'USA': 9,
    'Japan': 10,
    'Mexico': 11,
    'NewZealand': 12,
    'IndependentStateofWesternSamoa': 13,
    'Philippines': 14,
    'VietNam': 15,
    'CostaRica': 16,
    'PapuaNewGuinea': 17,
    'Vanuatu': 18,
    'ElSalvador': 19,

}


In [59]:
row=data.iloc[0]

In [60]:
row

code              waka
location      Wakkanai
city        稚内港-北海道稚内市
country          Japan
lat              45.41
lon             141.69
Name: 0, dtype: object

In [63]:
row['country']

'Japan'

In [78]:
pd.notna(1)

True

In [79]:
def check_stand(country, cid, city, location):
    if pd.notna(country) and pd.notna(cid) and pd.notna(city) and pd.notna(location):
        return True
    else:
        return False


for index in range(num_rows-1):
    series_column = data.iloc[index]
    print(series_column)

    country_str: str = series_column['country']
    # 从字典中找到对应的id
    cid: int = dict_country.get(country_str, -1)
    location_str: str = series_column['location']
    city_str: str = series_column['city']
    if check_stand(country_str, cid, city_str, location_str):
        region_model = RegionInfo(
            val_en=location_str, val_ch=city_str, pid=cid)
        session.add(region_model)
# UnicodeEncodeError: 'charmap' codec can't encode characters in position 0-2: character maps to <undefined>
try:
    session.commit()
except Exception as ex:
    session.rollback()
    print(ex.args)
finally:
    session.close()


code              waka
location      Wakkanai
city        稚内港-北海道稚内市
country          Japan
lat              45.41
lon             141.69
Name: 0, dtype: object
code              abas
location      Abashiri
city        网走港-北海道网走市
country          Japan
lat              44.02
lon             144.29
Name: 1, dtype: object
code            hana
location    Hanasaki
city          北海道根室市
country        Japan
lat            43.28
lon           145.57
Name: 2, dtype: object
code           kush
location    Kushiro
city         北海道钏路市
country       Japan
lat           42.98
lon          144.37
Name: 3, dtype: object
code            hako
location    Hakodate
city          北海道函馆市
country        Japan
lat            41.78
lon           140.72
Name: 4, dtype: object
code           fuka
location    Fukaura
city             深浦
country       Japan
lat           40.65
lon          139.93
Name: 5, dtype: object
code           ofun
location    Ofunato
city           大船渡港
country       Japan
lat           

In [82]:
# step3:录入station信息
def check_stand_station(code, location, lat, lon):
    if pd.notna(code) and pd.notna(location) and pd.notna(lat) and pd.notna(lon):
        return True
    else:
        return False


def get_rid(location: str) -> int:
    """
        根据 location 从 region_info 表中获取对应的rid
    """
    query = select(RegionInfo).where(RegionInfo.val_en == location)
    filter = session.scalars(query).fetchall()
    rid: int = -1
    if len(filter) > 0:
        # 取出第一个
        find_one = filter[0]
        rid = find_one.id
        pass
    return rid
    pass


for index in range(num_rows-1):
    series_column = data.iloc[index]
    print(series_column)
    code: str = series_column['code']
    location_str: str = series_column['location']
    lat: float = series_column['lat']
    lon: float = series_column['lon']
    if check_stand_station(code, location_str, lat, lon):
        rid: int = get_rid(location_str)
        station_model = StationInfo(
            station_code=code, lat=lat, lon=lon, rid=rid)
        session.add(station_model)
# UnicodeEncodeError: 'charmap' codec can't encode characters in position 0-2: character maps to <undefined>
try:
    session.commit()
except Exception as ex:
    session.rollback()
    print(ex.args)
finally:
    session.close()


code              waka
location      Wakkanai
city        稚内港-北海道稚内市
country          Japan
lat              45.41
lon             141.69
Name: 0, dtype: object
code              abas
location      Abashiri
city        网走港-北海道网走市
country          Japan
lat              44.02
lon             144.29
Name: 1, dtype: object
code            hana
location    Hanasaki
city          北海道根室市
country        Japan
lat            43.28
lon           145.57
Name: 2, dtype: object
code           kush
location    Kushiro
city         北海道钏路市
country       Japan
lat           42.98
lon          144.37
Name: 3, dtype: object
code            hako
location    Hakodate
city          北海道函馆市
country        Japan
lat            41.78
lon           140.72
Name: 4, dtype: object
code           fuka
location    Fukaura
city             深浦
country       Japan
lat           40.65
lon          139.93
Name: 5, dtype: object
code           ofun
location    Ofunato
city           大船渡港
country       Japan
lat           